In [ ]:
%cd ./../

In [ ]:
import logging
import os

from dotenv import load_dotenv

load_dotenv()
os.environ['INDEX'] = str(0)

# TODO: share the code with the `logs.py` file.

logging.basicConfig(
	format="{name}\t{asctime}\t{levelname}\t{message}\t",
	style='{',
	level=logging.DEBUG
)
logging.getLogger('PIL.PngImagePlugin').disabled = True
logging.getLogger('matplotlib').disabled = True
logging.getLogger('matplotlib.font_manager').disabled = True

In [ ]:
import repro  # Imported for its side effects.

In [ ]:
# noinspection PyUnresolvedReferences
from data import train_data_loader, test_data_loader, train_eval_data_loader
from model import model
from torch.nn import CrossEntropyLoss
import optim
from finetunes.lr import MyLrFinder as LRFinder

criterion = CrossEntropyLoss()  # TODO: move into a module to share with the `main` module.
optimizer = optim.init()

lr_finder = LRFinder(model=model, optimizer=optimizer, criterion=criterion)

In [ ]:
import conf

try:
	lr_finder.range_test(
		train_loader=train_data_loader,
		# val_loader=train_eval_data_loader,
		# val_loader=test_data_loader, # Data leakage warning!
		start_lr=1e-7, end_lr=0.5,
		num_iter=len(train_data_loader) * conf.EPOCHS * 1,
		# num_iter=conf.ROUNDS, epochs_per_iter=conf.EPOCHS,
		# reset_per_iter=True,
		smooth_f=0.05, diverge_th=5
	)
finally:
	lr_finder.reset()

In [ ]:
import pandas as pd
from finetunes.lr import find_lr_steep_grad
import plotly.express as px

df = [{'lr': lr, 'loss': loss} for lr, loss in zip(lr_finder.history['lr'], lr_finder.history['loss'])]
df = pd.DataFrame(df)

fig = px.line(
	df,
	x='lr', y='loss', log_x=True,
	title='Loss vs. Learning Rate', labels={'lr': 'Learning Rate', 'loss': 'Loss'},
)

steep_grad_lr, steep_grad_loss = find_lr_steep_grad(lr_finder)
fig.add_scatter(x=(steep_grad_lr,), y=(steep_grad_loss,), name='Steep. Grad.')

fig.show()